In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

class Classifier(nn.Module):

    # 모델의 코드는 여기서 작성해주세요

    def __init__(self, drop_prob):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(32*32*3, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(drop_prob)
        self.activation = nn.Sigmoid()

        self.bn1 = nn.BatchNorm1d(256) # argument : 1 layer output dimension
        self.bn2 = nn.BatchNorm1d(128) # argument : 2 layer output dimension

    def forward(self, x):
        z1 = self.linear1(x)
        z1 = self.bn1(z1)              # batch normalization 적용
        a1 = self.activation(z1)
        a1 = self.dropout(a1)

        z2 = self.linear2(a1)
        z2 = self.bn2(z2)              # batch normalization 적용
        a2 = self.activation(z2)
        a2 = self.dropout(a2)
        
        output = self.linear3(a2)

        return output


if __name__ == "__main__":

    train_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                 train=True,
                                                 transform=transforms.ToTensor(),
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                train=False,
                                                transform=transforms.ToTensor(),
                                                download=True)
    
    batch_size = 128

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

    model = Classifier(0.3).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999)) 
    criterion = nn.CrossEntropyLoss()

    epochs = 5
    lmbd = 0.003

    total_batch_num = len(train_dataloader)

    for epoch in range(epochs):
      avg_cost = 0
      model.train()

      for b_x, b_y in train_dataloader:
        b_x = b_x.view(-1, 32*32*3).to(device)
        logits = model(b_x) # forward prop
        loss = criterion(logits, b_y.to(device)) # get cost

        reg = model.linear1.weight.pow(2.0).sum()
        reg += model.linear2.weight.pow(2.0).sum()
        reg += model.linear3.weight.pow(2.0).sum()

        loss += lmbd*reg/len(b_x)/len(b_x)/2.

        optimizer.zero_grad()
        loss.backward() # backward prop
        optimizer.step() # update parameters

        avg_cost += loss / total_batch_num

      print('Epoch : {} / {}, cost : {}'.format(epoch+1, epochs, avg_cost))

    torch.save(model.state_dict(), 'model.pt')  # 학습된 모델을 저장하는 코드입니다.


Files already downloaded and verified
Files already downloaded and verified
Epoch : 1 / 5, cost : 1.9185748100280762
Epoch : 2 / 5, cost : 1.8009995222091675
Epoch : 3 / 5, cost : 1.7615118026733398
Epoch : 4 / 5, cost : 1.7291324138641357


In [4]:
# 학습된 모델의 성능을 평가하는 코드입니다.
# 아래의 코드로 평가를 진행할 예정이므로 아래의 코드가 정상 동작 해야하며, 제출전 모델의 성능을 확인하시면 됩니다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

classifier = Classifier(0.3).to(device)
classifier.load_state_dict(torch.load('model.pt'))
classifier.eval()


for data, label in test_dataloader:
    data = data.view(-1, 32 * 32 * 3).to(device)

    with torch.no_grad():
        logits = classifier(data)

        pred = torch.argmax(logits, dim=1)

        total = len(label)
        correct = torch.eq(pred, label.to(device)).sum()

        print("Accuracy on test set : {:.4f}%".format(100 * correct / total))

Files already downloaded and verified
Accuracy on test set : 40.5000%
